In [1]:
import pandas as pd

In [2]:
df_questions = pd.read_csv("../datasets/cleaned/cleaned_dataset.csv")
df_documents = pd.read_csv("../datasets/cleaned/cleaned_document_dataset.csv")

In [ ]:
import dspy
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from dspy.retrieve.qdrant_rm import QdrantRM
from vectorizer import CustomSentenceVectorizer

In [213]:
encoder = SentenceTransformer('intfloat/multilingual-e5-large')

In [6]:
qdrant = QdrantClient(":memory:")

In [7]:
# Create collection to store books
qdrant.recreate_collection(
    collection_name="main-document",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_17154/1864381341.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [65]:
qdrant.upload_records(
    collection_name="main-document",
    records=[
        models.Record(
            id=idx,
            vector=eval(record["embedding"]),
            payload={"sentence": record["sentence"]}
        ) for idx, record in df_documents.iterrows()
    ]
)

/tmp/ipykernel_17154/2208216076.py:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant.upload_records(


In [66]:
e5_vectorizer = CustomSentenceVectorizer('intfloat/multilingual-e5-large')

In [67]:
qdrant_retriever_model = QdrantRM("main-document", qdrant, k=3, vectorizer=e5_vectorizer, document_field="sentence")

In [204]:
ollama_model = dspy.OllamaLocal(model="command-r",model_type='text',
                                max_tokens=10000,
                                temperature=0.1,
                                top_p=0.8, frequency_penalty=1.17, top_k=40)

In [206]:
dspy.settings.configure(lm=ollama_model)

In [207]:
class GenerateAnswer(dspy.Signature):
    """الإجابة على الأسئلة بإجابات مختصرة ومباشرة."""

    context = dspy.InputField(prefix="السياق:", desc="من الممكن ان يحتوي على السياق")
    question = dspy.InputField(prefix="السؤال:", desc=":{السؤال}")
    answer = dspy.OutputField(prefix="الجواب:", desc="جواب من ٥ الى ١٠ كلمات")

In [208]:
arabic_rationale = dspy.OutputField(
            prefix="الاستدلال: دعونا نفكر خطوة بخطوة من أجل",
            desc="{إنتاج الإجابة}. نقوم ب…",
        )

In [209]:
class RAG(dspy.Module):
    def __init__(self, num_passages=1):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer, rationale_type=arabic_rationale)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [210]:
uncompiled_rag = RAG()

In [211]:
my_question = "ما هي خطة الانفاق للأعوام القادمة؟"
response = uncompiled_rag(my_question)
print(response.answer)

زيادة الإنفاق على المشاريع التنموية.


In [212]:
ollama_model.inspect_history(n=1)




الإجابة على الأسئلة بإجابات مختصرة ومباشرة.

---

Follow the following format.

السياق: من الممكن ان يحتوي على السياق

السؤال: :{السؤال}

الاستدلال: دعونا نفكر خطوة بخطوة من أجل {إنتاج الإجابة}. نقوم ب…

الجواب: جواب من ٥ الى ١٠ كلمات

---

السياق: «تعتزم الحكومة خلال العام القادم وعلى المدى المتوسط عبر سياساتها المالية على التوسع في مستويات الإنفاق الموجّه بشكل رئيسي إلى البرامج والمشاريع التنموية والاستراتيجيات المناطقية والقطاعية الدافعة لتعزيز النمو الاقتصادي المستدام، بالإضافة إلى تحسين ورفع جودة الخدمات العامة وتطوير البنية التحتية في مختلف مناطق المملكة؛ مما يساهم في رفع جودة الحياة وفقًا لرؤية السعودية 2030، بالإضافة إلى تمكين القطاع الخاص وتحفيز البيئة الاستثمارية، والاستمرار في تحسين منظومة الدعم والإعانات الاجتماعية. كما»

السؤال: ما هي خطة الانفاق للأعوام القادمة؟

الاستدلال: دعونا نفكر خطوة بخطوة من أجل زيادة الإنفاق على المشاريع التنموية والاستراتيجيات القطاعية لتعزيز النمو الاقتصادي.

الجواب: زيادة الإنفاق على المشاريع التنموية.





'\n\n\nالإجابة على الأسئلة بإجابات مختصرة ومباشرة.\n\n---\n\nFollow the following format.\n\nالسياق: من الممكن ان يحتوي على السياق\n\nالسؤال: :{السؤال}\n\nالاستدلال: دعونا نفكر خطوة بخطوة من أجل {إنتاج الإجابة}. نقوم ب…\n\nالجواب: جواب من ٥ الى ١٠ كلمات\n\n---\n\nالسياق: «تعتزم الحكومة خلال العام القادم وعلى المدى المتوسط عبر سياساتها المالية على التوسع في مستويات الإنفاق الموجّه بشكل رئيسي إلى البرامج والمشاريع التنموية والاستراتيجيات المناطقية والقطاعية الدافعة لتعزيز النمو الاقتصادي المستدام، بالإضافة إلى تحسين ورفع جودة الخدمات العامة وتطوير البنية التحتية في مختلف مناطق المملكة؛ مما يساهم في رفع جودة الحياة وفقًا لرؤية السعودية 2030، بالإضافة إلى تمكين القطاع الخاص وتحفيز البيئة الاستثمارية، والاستمرار في تحسين منظومة الدعم والإعانات الاجتماعية. كما»\n\nالسؤال: ما هي خطة الانفاق للأعوام القادمة؟\n\nالاستدلال: دعونا نفكر خطوة بخطوة من أجل زيادة الإنفاق على المشاريع التنموية والاستراتيجيات القطاعية لتعزيز النمو الاقتصادي.\n\nالجواب:\x1b زيادة الإنفاق على المشاريع التنموية.\x1b\n\n\n